In [1]:
from mrmr_ae.ProgMAS import ProgMAS
import pandas as pd

In [2]:
survive_data=pd.read_csv('data\survive_BLCA.csv'
                         ,index_col=0,sep=',')
survive_data=survive_data.dropna(how='any')

In [3]:
import mrmr_ae
def add_survival_labels(data):
    data['label'] = 0
    data.loc[(data['OS.time'] > 1825) & (data['OS'].isin([0, 1])), 'label'] = 1
    data = data.loc[~((data['OS.time'] < 1825) & (data['OS'] == 0))]
    return data
survive_data=add_survival_labels(survive_data)


In [4]:
CNV_data=pd.read_table('data\CNV_BLCA'
                         ,index_col=0)
RNA_data=pd.read_table('data\RNA_BLCA'
                         ,index_col=0)
miRNA_data=pd.read_table('data\RPPA'
                         ,index_col=0)

In [5]:
omics_data = {
    'omics1': CNV_data,
    'omics2': RNA_data,
    'omics3': miRNA_data
}



In [6]:
from mrmr_ae.Preprocess import MultiomicsDataProcessor


In [7]:
processor=MultiomicsDataProcessor()
missing_thresholds = {
    'omics1': 0.2,
    'omics2': 0.2,
    'omics3': 0.2
}

variance_thresholds = {
    'omics1': 0.2,
    'omics2': 0.2,
    'omics3': 0.2
}
processor.process_multiomics_data(survive_data, omics_data, missing_thresholds, variance_thresholds)

In [8]:
survival_data_processed = processor.survival_data
omics1_processed = processor.get_processed_data('omics1')
omics2_processed = processor.get_processed_data('omics2')
omics3_processed = processor.get_processed_data('omics3')


In [9]:
model_params = {'kernel': 'linear', 'C': 0.1,'gamma':10}
ae_params = {
        'epochs': 300,
        'batch_size': 16
    }
mrmr_params = {
        'k': 100,
        'p_dict': {
            'omics1': 1,
            'omics2': 1,
            'omics3': 1
        }
    }

progmas = ProgMAS(ae_params=ae_params, mrmr_params=mrmr_params)


In [10]:
   # Step 2: Set random seed (optional)
progmas.set_seed(529)


In [11]:
    # Step 3: Get latent representation using get_latent_repre method
omics_dict = {
        'omics1': omics1_processed,
        'omics2': omics2_processed,
        'omics3': omics3_processed
    }

In [12]:
from mrmr_ae.multi_mrmr import Multi_mRMR

In [13]:
merged_omics, embedded_Z = progmas.get_latent_repre(omics_dict, survival_data_processed)


omics-specific mRMR begin


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:34<00:00,  2.87it/s]


Selected 100 features from omics1


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:40<00:00,  2.20s/it]


Selected 100 features from omics2


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:06<00:00, 14.31it/s]


Selected 100 features from omics3


omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
14/14 [==============================] - 1s 2ms/step - loss: 0.1252
Epoch 2/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1177
Epoch 3/300
14/14 [==============================] - 0s 2ms/step - loss: 0.1113
Epoch 4/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1050
Epoch 5/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0973
Epoch 6/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0879
Epoch 7/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0758
Epoch 8/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0604
Epoch 9/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0453
Epoch 10/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0345
Epoch 11/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0283
Epoch 12/300
14/14 [==============================] - 0s 1ms/step - los

14/14 [==============================] - 0s 1ms/step - loss: 0.0183
Epoch 202/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0184
Epoch 203/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0183
Epoch 204/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0182
Epoch 205/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0182
Epoch 206/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0181
Epoch 207/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 208/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0181
Epoch 209/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0183
Epoch 210/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0182
Epoch 211/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 212/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 213/300
14/14 [=========

In [14]:
    # Step 4: Split data into training and testing sets
progmas.split_train_test(survival_data_processed, test_size=0.3)


In [15]:
    # Step 5: Train and predict using SVM model

acc, auc, f1 = progmas.train_predict(model='svm', model_params=model_params)
print("SVM Model - AUC: {:.4f}, Accuracy: {:.4f}, F1-Score: {:.4f}".format(auc, acc, f1))


SVM Model - AUC: 0.9187, Accuracy: 0.8641, F1-Score: 0.8474


In [16]:
    # Step 6: Perform survival analysis
p_values = progmas.survival_analysis()
print("Log-Rank Test - p-values:\n", p_values)


Log-Rank Test - p-values:
 0.021574255343144354


In [20]:
    # Step 7: Perform biomarker identification
biomarker_results = progmas.biomarker_idt(omics_dict, survival_data_processed)
for omics_name, top_features in biomarker_results.items():
 print("Top features for {}: {}".format(omics_name, top_features))

omics-specific mRMR begin


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.34it/s]


Selected 100 features from omics1
omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
14/14 [==============================] - 1s 1ms/step - loss: 0.1046
Epoch 2/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0733
Epoch 3/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0638
Epoch 4/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0584
Epoch 5/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0551
Epoch 6/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0529
Epoch 7/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0493
Epoch 8/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0470
Epoch 9/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0442
Epoch 10/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0414
Epoch 11/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0386
Epoch 12/300
14/14 [=================

14/14 [==============================] - 0s 1ms/step - loss: 0.0073
Epoch 102/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0079
Epoch 103/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0075
Epoch 104/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0074
Epoch 105/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0078
Epoch 106/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0076
Epoch 107/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0072
Epoch 108/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0079
Epoch 109/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0075
Epoch 110/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0085
Epoch 111/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0073
Epoch 112/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0079
Epoch 113/300
14/14 [=========

14/14 [==============================] - 0s 1ms/step - loss: 0.0065
Epoch 202/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 203/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 204/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0065
Epoch 205/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 206/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0061
Epoch 207/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 208/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0065
Epoch 209/300
14/14 [==============================] - 0s 3ms/step - loss: 0.0062
Epoch 210/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 211/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 212/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0067
Epoch 213/300
14/14 [=========

...

Selected 100 features from omics2
omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
14/14 [==============================] - 0s 622us/step - loss: 0.1951
Epoch 2/300
14/14 [==============================] - 0s 578us/step - loss: 0.1628
Epoch 3/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1510
Epoch 4/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1423
Epoch 5/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1355
Epoch 6/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1295
Epoch 7/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1236
Epoch 8/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1168
Epoch 9/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1103
Epoch 10/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1023
Epoch 11/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0945
Epoch 12/300
14/14 [=============

14/14 [==============================] - 0s 2ms/step - loss: 0.0167
Epoch 202/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0167
Epoch 203/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0166
Epoch 204/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0168
Epoch 205/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0165
Epoch 206/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0169
Epoch 207/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0164
Epoch 208/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0169
Epoch 209/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0164
Epoch 210/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0164
Epoch 211/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0167
Epoch 212/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0164
Epoch 213/300
14/14 [=========

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 14.16it/s]


Selected 100 features from omics3
omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
14/14 [==============================] - 1s 2ms/step - loss: 0.1741
Epoch 2/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1557
Epoch 3/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1458
Epoch 4/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1378
Epoch 5/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1311
Epoch 6/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1229
Epoch 7/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1145
Epoch 8/300
14/14 [==============================] - 0s 1ms/step - loss: 0.1041
Epoch 9/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0930
Epoch 10/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0807
Epoch 11/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0690
Epoch 12/300
14/14 [=================

14/14 [==============================] - 0s 2ms/step - loss: 0.0120
Epoch 202/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0116
Epoch 203/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0117
Epoch 204/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0116
Epoch 205/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0119
Epoch 206/300
14/14 [==============================] - 0s 2ms/step - loss: 0.0119
Epoch 207/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0118
Epoch 208/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0120
Epoch 209/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0117
Epoch 210/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0118
Epoch 211/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0117
Epoch 212/300
14/14 [==============================] - 0s 1ms/step - loss: 0.0118
Epoch 213/300
14/14 [=========

# CNV RNA RPPA

In [21]:
CNV_data=pd.read_table('E:\自己文件存放\分期\膀胱癌\gz\copy'
                         ,index_col=0)
RNA_data=pd.read_table('E:\自己文件存放\分期\膀胱癌\gz\RNA'
                         ,index_col=0)
RPPA_data=pd.read_table('E:\自己文件存放\分期\膀胱癌\gz\RPPA'
                         ,index_col=0)

In [22]:
omics_data = {
    'omics1': CNV_data,
    'omics2': RNA_data,
    'omics3': RPPA_data
}


In [23]:
from mrmr_ae.Preprocess import MultiomicsDataProcessor


In [24]:
processor=MultiomicsDataProcessor()
missing_thresholds = {
    'omics1': 0.2,
    'omics2': 0.2,
    'omics3': 0.2
}

variance_thresholds = {
    'omics1': 0.001,
    'omics2': 0.2,
    'omics3': 0
}


In [25]:
processor.process_multiomics_data(survive_data, omics_data, missing_thresholds, variance_thresholds)

In [26]:
survival_data_processed = processor.survival_data
omics1_processed = processor.get_processed_data('omics1')
omics2_processed = processor.get_processed_data('omics2')
omics3_processed = processor.get_processed_data('omics3')

In [27]:
model_params = {'kernel': 'linear', 'C': 0.1,'gamma':10}
ae_params = {
        'epochs': 300,
        'batch_size': 16
    }
mrmr_params = {
        'k': 100,
        'p_dict': {
            'omics1': 1,
            'omics2': 1,
            'omics3': 1
        }
    }
progmas = ProgMAS(ae_params=ae_params, mrmr_params=mrmr_params)


In [28]:
    # Step 2: Set random seed (optional)
progmas.set_seed(529)


In [29]:
    # Step 3: Get latent representation using get_latent_repre method
omics_dict = {
        'omics1': omics1_processed,
        'omics2': omics2_processed,
        'omics3': omics3_processed
    }
merged_omics, embedded_Z = progmas.get_latent_repre(omics_dict, survival_data_processed)


omics-specific mRMR begin


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.34it/s]


Selected 100 features from omics1


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [03:34<00:00,  2.15s/it]


Selected 100 features from omics2


100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:06<00:00, 10.58it/s]


Selected 72 features from omics3
omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
12/12 [==============================] - 1s 1ms/step - loss: 0.1344
Epoch 2/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1275
Epoch 3/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1218
Epoch 4/300
12/12 [==============================] - 0s 1ms/step - loss: 0.1165
Epoch 5/300
12/12 [==============================] - 0s 2ms/step - loss: 0.1110
Epoch 6/300
12/12 [==============================] - 0s 1ms/step - loss: 0.1047
Epoch 7/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0979
Epoch 8/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0899
Epoch 9/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0795
Epoch 10/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0679
Epoch 11/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0555
Epoch 12/300
12/12 [==================

12/12 [==============================] - 0s 2ms/step - loss: 0.0164
Epoch 202/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0163
Epoch 203/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0164
Epoch 204/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0165
Epoch 205/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0168
Epoch 206/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0165
Epoch 207/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0166
Epoch 208/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0165
Epoch 209/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0165
Epoch 210/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0165
Epoch 211/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0164
Epoch 212/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0164
Epoch 213/300
12/12 [=========

In [30]:
    # Step 4: Split data into training and testing sets
progmas.split_train_test(survival_data_processed, test_size=0.3)


In [31]:
    # Step 5: Train and predict using SVM model

acc, auc, f1 = progmas.train_predict(model='svm', model_params=model_params)
print("SVM Model - AUC: {:.4f}, Accuracy: {:.4f}, F1-Score: {:.4f}".format(auc, acc, f1))


SVM Model - AUC: 0.9520, Accuracy: 0.9119, F1-Score: 0.9047


In [32]:
    # Step 6: Perform survival analysis
p_values = progmas.survival_analysis()
print("Log-Rank Test - p-values:\n", p_values)


Log-Rank Test - p-values:
 0.00045726387269152307


In [33]:
    # Step 7: Perform biomarker identification
biomarker_results = progmas.biomarker_idt(omics_dict, survival_data_processed)
for omics_name, top_features in biomarker_results.items():
 print("Top features for {}: {}".format(omics_name, top_features))

omics-specific mRMR begin


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:36<00:00,  2.72it/s]


Selected 100 features from omics1
omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
12/12 [==============================] - 1s 1ms/step - loss: 0.1032
Epoch 2/300
12/12 [==============================] - 0s 921us/step - loss: 0.0777
Epoch 3/300
12/12 [==============================] - 0s 767us/step - loss: 0.0665
Epoch 4/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0590
Epoch 5/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0549
Epoch 6/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0522
Epoch 7/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0497
Epoch 8/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0476
Epoch 9/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0449
Epoch 10/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0422
Epoch 11/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0401
Epoch 12/300
12/12 [=============

12/12 [==============================] - 0s 1ms/step - loss: 0.0085
Epoch 102/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 103/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0082
Epoch 104/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0086
Epoch 105/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0083
Epoch 106/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 107/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0086
Epoch 108/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0084
Epoch 109/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 110/300
12/12 [==============================] - 0s 414us/step - loss: 0.0085
Epoch 111/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0081
Epoch 112/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0079
Epoch 113/300
12/12 [=======

12/12 [==============================] - 0s 1ms/step - loss: 0.0078
Epoch 202/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0077
Epoch 203/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0078
Epoch 204/300
12/12 [==============================] - 0s 732us/step - loss: 0.0076
Epoch 205/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0075
Epoch 206/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0078
Epoch 207/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0076
Epoch 208/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0076
Epoch 209/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0075
Epoch 210/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0077
Epoch 211/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0078
Epoch 212/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0079
Epoch 213/300
12/12 [=======

...

12/12 [==============================] - 0s 2ms/step - loss: 0.0178
Epoch 202/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0178
Epoch 203/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0177
Epoch 204/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0183
Epoch 205/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0182
Epoch 206/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0178
Epoch 207/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0179
Epoch 208/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0181
Epoch 209/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0179
Epoch 210/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0178
Epoch 211/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0181
Epoch 212/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 213/300
12/12 [=========

100%|██████████████████████████████████████████████████████████████████████████████████| 72/72 [00:05<00:00, 12.02it/s]


Selected 72 features from omics3
omics-specific mRMR end
Auto-encoder has ready
Epoch 1/300
12/12 [==============================] - 0s 944us/step - loss: 0.2361
Epoch 2/300
12/12 [==============================] - 0s 1ms/step - loss: 0.2078
Epoch 3/300
12/12 [==============================] - 0s 1ms/step - loss: 0.1954
Epoch 4/300
12/12 [==============================] - 0s 0s/step - loss: 0.1872
Epoch 5/300
12/12 [==============================] - 0s 0s/step - loss: 0.1797
Epoch 6/300
12/12 [==============================] - 0s 223us/step - loss: 0.1714
Epoch 7/300
12/12 [==============================] - 0s 623us/step - loss: 0.1634
Epoch 8/300
12/12 [==============================] - 0s 233us/step - loss: 0.1541
Epoch 9/300
12/12 [==============================] - 0s 225us/step - loss: 0.1445
Epoch 10/300
12/12 [==============================] - 0s 139us/step - loss: 0.1321
Epoch 11/300
12/12 [==============================] - 0s 251us/step - loss: 0.1192
Epoch 12/300
12/12 [======

Epoch 101/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0141
Epoch 102/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0142
Epoch 103/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0146
Epoch 104/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0139
Epoch 105/300
12/12 [==============================] - 0s 2ms/step - loss: 0.0142
Epoch 106/300
12/12 [==============================] - 0s 384us/step - loss: 0.0138
Epoch 107/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0141
Epoch 108/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0138
Epoch 109/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0139
Epoch 110/300
12/12 [==============================] - 0s 1ms/step - loss: 0.0140
Epoch 111/300
12/12 [==============================] - 0s 763us/step - loss: 0.0139
Epoch 112/300
12/12 [==============================] - 0s 284us/step - loss: 0.0143
Epoch 113/

Epoch 300/300
6/6 [==============================] - 0s 0s/step
Auto-encoder end
Top features for omics1: ['LRRC16A', 'CCDC171', 'LINC00907', 'PRKG1', 'CTTN', 'PRDX6', 'JPH2', 'EEA1', 'POLB', 'RN7SL147P']
Top features for omics2: ['POC1B', 'EIF2C3', 'DSCR10', 'SLC5A4', 'SGMS1', 'KCNQ2', 'LYPD4', 'DHCR7', 'NEU4', 'PEX2']
Top features for omics3: ['Src-M-V', 'eEF2K-R-V', 'S6_pS235_S236-R-V', 'Caspase-7_cleavedD198-R-C', 'G6PD-M-V', 'Annexin-1-M-E', 'PKC-alpha-M-V', 'eIF4G-R-C', 'ASNS-R-V', 'GATA3-M-V']
